In [3]:
# Prepare data for tokenizer
!python3 tokenizer/prepare-visfd.py \
    ./data/Train.csv \
    --out_dir ./tokenizer

2024-05-05 17:59:40 INFO  WordSegmenter:24 - Loading Word Segmentation model
2024-05-05 17:59:40 INFO  PosTagger:23 - Loading POS Tagging model
2024-05-05 17:59:42 INFO  NerRecognizer:34 - Loading NER model
2024-05-05 17:59:50 INFO  DependencyParser:32 - Loading Dependency Parsing model
Progress: 100%|███████████████████████████▉| 7783/7786 [00:35<00:00, 221.42it/s]


In [4]:
# Build tokenizer
!python3 tokenizer/build-tokenizer.py \
    ./tokenizer/feedbacks.txt \
    --out_dir ./tokenizer \
    --config_file ./config.json

[00:00:00] Tokenize words                 ██████████████████ 7688     /     7688[00:00:00] Tokenize words                 ██████████████████ 0        /        0
[00:00:00] Count pairs                    ██████████████████ 7688     /     7688
[00:00:00] Compute merges                 ██████████████████ 5640     /     5640


In [1]:
from tokenizers import Tokenizer
from modeling.modules import ViSFD_LSTM
from modeling.losses import ViSFDLoss
from modeling.dataset import ViSFDDataset
import torch

task = "mtl"

train_set = ViSFDDataset("data/Train.csv", task_type=task)
val_set = ViSFDDataset("data/Dev.csv", task_type=task)
test_set = ViSFDDataset("data/Test.csv", task_type=task)

2024-05-05 20:28:14 INFO  WordSegmenter:24 - Loading Word Segmentation model
2024-05-05 20:28:15 INFO  PosTagger:23 - Loading POS Tagging model
2024-05-05 20:28:16 INFO  NerRecognizer:34 - Loading NER model
2024-05-05 20:28:24 INFO  DependencyParser:32 - Loading Dependency Parsing model


100%|██████████| 2224/2224 [00:00<00:00, 2302.52it/s]


In [2]:
tokenizer: Tokenizer = Tokenizer.from_file("./tokenizer/tokenizer.json")
model = ViSFD_LSTM(
    tokenizer, 
    task_type=task, 
    embed_dim=768,
    dropout=0.5,
    lstm_hidden_size=512,
    cnn_kernel_size=5,
    cnn_out_channels=16,
    pooling_out_size=64,
    output_dropout=0.5,
    output_hidden_size=64
)

from torchinfo import summary
summary(model)

/home/ptdat/Desktop/venv/lib/python3.12/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Layer (type:depth-idx)                   Param #
ViSFD_LSTM                               --
├─Embedding: 1-1                         4,608,000
├─LSTM: 1-2                              5,251,072
├─Conv1d: 1-3                            81,936
├─AdaptiveAvgPool1d: 1-4                 --
├─AdaptiveMaxPool1d: 1-5                 --
├─MTL_ViSFDClassifier: 1-6               --
│    └─AspectClassifier: 2-1             --
│    │    └─Linear: 3-1                  22,539
│    └─PolarityClassifier: 2-2           --
│    │    └─ModuleList: 3-2              61,470
Total params: 10,025,017
Trainable params: 10,025,017
Non-trainable params: 0

In [4]:
from modeling.utils import train
from modeling.metrics import AspectF1Score

train(
    model=model,
    loss_fn=ViSFDLoss(task_type=task),
    optimizer=torch.optim.AdamW(model.parameters()),
    train_set=train_set,
    validation_set=val_set,
    batch_size=32,
    epochs=5,
    metrics={
        "Aspect F1": AspectF1Score(task_type=task, aspect_threshold=0.4)
    },
    device="cuda"
)

Epoch 1 | Training Loss 0.4532: 100%|██████████| 244/244 [00:15<00:00, 15.98it/s]
Loss Evaluation: 100%|██████████| 35/35 [00:00<00:00, 55.60it/s]
Epoch 2 | Training Loss 0.3515:  99%|█████████▉| 241/244 [00:13<00:00, 17.79it/s]/home/ptdat/Desktop/venv/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Epoch 2 | Training Loss 0.5676: 100%|██████████| 244/244 [00:13<00:00, 18.16it/s]
Loss Evaluation: 100%|██████████| 35/35 [00:00<00:00, 56.83it/s]
Epoch 3 | Training Loss 0.1598: 100%|██████████| 244/244 [00:13<00:00, 18.02it/s]
Loss Evaluation: 100%|██████████| 35/35 [00:00<00:00, 54.96it/s]
Epoch 4 | Training Loss 0.2939: 100%|██████████| 244/244

In [16]:
from torch.utils.data import DataLoader
from modeling.utils import stl_decode, mtl_decode

i = 27
# x, y = next(iter(DataLoader(val_set, 2)))
x, y = val_set[i]
model.eval()
y_hat = model(x)
mtl_decode(y_hat), val_set.data.label[i]

([{'CAMERA': 'Positive',
   'PRICE': 'Negative',
   'GENERAL': 'Positive',
   'PERFORMANCE': 'Positive',
   'DESIGN': 'Positive',
   'OTHERS': ''}],
 '{CAMERA#Positive};{DESIGN#Positive};{PRICE#Neutral};{GENERAL#Positive};{OTHERS};')